In [2]:
from time import time
import os
import collections

In [3]:
def def_onlyCode(allLines):
    copyline = allLines.copy()
    newLines=[]
    lines=iter(allLines)
    citer=0
    withoutSpace = lambda x : x.replace(' ','')
    blankLine = lambda x : x=='\n' or x==''
    commWildcard = lambda x:x[0]=='#'
    commOneline = lambda x:(initComment(x) and endComment(x[3:]))
    initComment = lambda x:(x[:3]=='"""')
    endComment = lambda x:(x[-4:-1]=='"""')
    for line in lines:
        citer=citer+1
        newLine = withoutSpace(line)
        if blankLine(newLine):
            continue
        if commWildcard(newLine):
            continue
        if commOneline(newLine):
            continue
        if initComment(newLine):
            try:
                line = next(lines)
            except:
                pass
            for i in range(citer,len(copyline)):
                newLine = withoutSpace(line)
                endLine = citer+1==len(copyline)
                try:
                    line = next(lines)
                except:
                    pass
                if endComment(newLine) or endLine:
                    break
                else:
                    citer=citer+1
        newLines.append(line)   
    return newLines

In [4]:
def depFor(archivo):
    newLineas=archivo.copy()
    lineas=iter(archivo)
    citer=0
    withoutSpace = lambda x : x.replace(' ','')
    initFor = lambda x:(x[:3]=='for')
    endFor = lambda x:(x[-2:-1]==':')
    countSpace = lambda x:(x.index(x.replace(' ','')[0]))
    arrMax=[0]
    arr=[]
    for linea in lineas:
        citer=citer+1
        wordFor='for'
        wordEnd=':'
        space = countSpace(linea)
        patronFor=((initFor(withoutSpace(linea))) and (endFor(withoutSpace(linea))))
        if patronFor:
            newLines=[]
            for i in range(citer,len(newLineas)):
                endLine = citer+1==len(newLineas)
                forSpace = False
                if not endLine:
                    nextspaceLine = countSpace(newLineas[citer])
                    forSpace = nextspaceLine<=space
                else:
                    try:
                        newLines.append(next(lineas))
                    except:
                        pass
                if forSpace or endLine:
                    value,arr=depFor(newLines)
                    arrMax.append(value+1)
                    break
                else:
                    try:
                        newLines.append(next(lineas))
                        citer=citer+1
                    except:
                        pass
    return(max(arrMax),arrMax)

In [30]:
def librerias(lines):
    setlibrerias=set()
    wordImport= 'import'
    wordFrom= 'from'
    notElement={'import', 'from', ''}
    for line in lines:
        inImport=(wordImport in line)
        inFrom=(wordFrom in line)
        if inFrom and inImport:
            arrlibr = line[:line.index('import')].split(' ')
            setlibrerias = setlibrerias|set(arrlibr)
            continue
        if inImport:
            arrlibr =  line[:-1].split(' ')
            setlibrerias = setlibrerias|set(arrlibr)
            continue
    newSetelement= list(setlibrerias-notElement)
    newArrelement = [ element.split('.')[0] for element in newSetelement]
    return set(newArrelement)

In [22]:
def countDef(lines):
    wordDef='def'
    wordEnd=':'
    arrArg=[]
    for line in lines:
        if wordDef in line and wordEnd in line:
            argini=line.find("(")+1
            argend=line.find(")")
            arg=line[argini:argend].split(',')
            arrArg.append(len(arg))
    return arrArg

In [23]:
def duplicateLine(arrlines):
    newGroup=[]
    withoutSpace = lambda x : x.replace(' ','')
    lines=list(map(withoutSpace,arrlines))
    unionLine=lambda x : x.replace('[','').replace(',','').replace(']','')
    for i,line in enumerate(lines):
        newGroup.append(str(unionLine(str(lines[i:i+4]))))
    repetidos=collections.Counter(newGroup)
    duplicado = [1 for key in repetidos if repetidos[key]>=2]
    return (sum(duplicado)+3*bool(duplicado))

In [24]:
def returnFilespy(carpeta):
    newArray=[]
    for archivo in os.listdir(carpeta):
        if os.path.isdir(os.path.join(carpeta,archivo)):
            newArray = newArray+returnFilespy(os.path.join(carpeta,archivo))
        else:
            newArray.append(os.path.join(carpeta,archivo))
    filesPy = [filePy for filePy in newArray if filePy[-3:]=='.py' ]
    return filesPy

In [25]:
ruoteFlies = '.\\Almacen_repositorios\\tensorflow-master'
filesPy = returnFilespy(ruoteFlies)
len(filesPy)

3411

In [31]:
start_time = time()
LinesCodetotal = 0
depForTotal = []
argDeftotal = []
libreryTotal =set()
duplicateTotal = 0
for filePy in filesPy:
#     print('********************************************************')
#     print(filePy)
    file = open(filePy, 'r', encoding="utf8")
    lines = file.readlines()
    
    linesCode = def_onlyCode(lines)
    LinesCodetotal=LinesCodetotal+len(linesCode)
#     print(len(linesCode))
    
    countFor = depFor(linesCode)[1][1:]
    depForTotal = depForTotal+countFor
#     print(countFor)
    
    libreriasSet = librerias(linesCode)
    libreryTotal = libreryTotal|libreriasSet
#     print(libreriasSet)
    
    countArgdef = countDef(linesCode)
    argDeftotal = argDeftotal+countArgdef
#     print(countArgdef)
    
    isDupplicate = duplicateLine(linesCode)
    duplicateTotal=duplicateTotal+isDupplicate
    
    file.close()
    
avgFor=sum(depForTotal)/len(depForTotal)
perctDuplicate = duplicateTotal/LinesCodetotal*100
avgArg = sum(argDeftotal)/len(argDeftotal)

print('____________________________________________________')
print()
print('number of lines : ' +str(LinesCodetotal))
print('libraries : '+str(list(libreryTotal)))
print('nesting factor : '+str(avgFor))
print('code duplication : '+str(perctDuplicate))
print('average variables : '+str(avgArg))

elapsed_time = time() - start_time
print(elapsed_time)

____________________________________________________

number of lines : 696194
libraries : ['', 'mark_as_used=False)', '_contextlib', '3]))', '_import_and_infer(', 'npr', 'submodules', 'new_value', 'test_op', 'TF-TRT', '4', 'prob', 'If', "help='Base", 'disable=wildcard-import', 'na_values="?")', 'import_event(tensor,', 'np', 'third_path,', 'Fs(20),', '_init_from_proto(self,', 'summary_ops_v2', 'restored_fullargspec', 'has', 'new_saver_3', 'tf_saver', 'output_op', 'symbol_id', 'open(self', '3]),', '%', 'tempfile', '"b"]),', 'op_index,', "'problem", '3', 'functools', 'hashlib', 'imported({"a":', 'inspect_utils', 'name="")', 'psutil', 'expected_', 'special', 'b_1', 'api_name):', '\\n")', 'Necessary', 'traceback', 'Do', '["tag_name"],', 'site;', 'saver_lib', 'input_layer', 'import_scope="scope_name")', '!=', '_init_values_from_proto(self,', 'imported_concrete,', '"Feature', 'import_scope="bar")', 'summary', 'import_scope="new_hidden1/new_hidden2",', 'import_scope="subgraph_1")', 'g', 'mars

In [27]:
start_time = time()
# for filePy in filesPy
#open file
file = open(r'.\Almacen_repositorios\tensorflow-master\configure.py', 'r')
lines = file.readlines()

##Prosesing file

#Lines of code
linesCode = def_onlyCode(lines)
# for line in linesCode:
#     print(line)
print(len(linesCode))

countFor = depFor(linesCode)
print(countFor[1][1:])

libreriasSet = librerias(linesCode)
print(libreriasSet)

countArgdef = countDef(linesCode)
print(countArgdef)

isDupplicate = duplicateLine(linesCode)
print(isDupplicate)

#close file
file.close()

elapsed_time = time() - start_time
print(elapsed_time)

1047
[1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]
{"'-c',", 'os', 'sys;', 'run_shell([python_bin_path,', '__future__', 're', 'argparse', 'distutils', 'platform', "'import", 'print(sys', 'site;', "'from", 'print("\\\\n"', 'subprocess', 'shutil', 'errno', 'sys', 'return'}
[1, 1, 1, 1, 1, 1, 2, 3, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1]
8
0.015962600708007812
